# <font color='Green'> Prediccion de Supervivencia en el Titanic </font>

In [1]:
##### Librerias ####
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
##### Importar los datos #####
test = pd.read_csv('C:/Users/Gonza/Documents/codigo/proyectos/Titanic/test.csv')
train = pd.read_csv('C:/Users/Gonza/Documents/codigo/proyectos/Titanic/train.csv')

## Analisis de los datos

In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* Ambos conjuntos tienen mismos datos (la de entrenamiento agrega columna 'Survived')

In [5]:
##### Verificacion de la cantidad de datos que hay en los dataset #####
print(train.shape)
print(test.shape)

# Columna de diferencia es la 'Survived'

(891, 12)
(418, 11)


In [6]:
##### Verificacion de los tipos de datos obtenidos en los dataset #####
print(train.info)
print(test.info)

<bound method DataFrame.info of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                   

In [7]:
##### Verificacion de datos faltantes en los dataset #####
print(pd.isnull(train).sum())
print(pd.isnull(test).sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [8]:
##### Verificacion de las estadisticas obtenidas en los dataset #####
print(train.describe())
print(test.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
       PassengerId      Pclass         Age       SibSp       Parch        Fare
co

## Tratamiento de los datos

* Cambio de formato de columna de sexo a numeros
* Cambio de formato de columna de embarque a numeros
* Remplazar los datos faltantes en la columna de edad por la media (Hace que no infiera)
* Agrupar por franjas la edad de los pasajeros
* Eliminacion de la columna 'Cabin'
* Eliminacion de las columnas que no se consideren necesarias
* Eliminacion de las filas con datos perdidos
* Verificacion

In [9]:
### Cambio de datos columna Sexo ###
train['Sex'].replace(['female','male'],[0,1],inplace=True)
test['Sex'].replace(['female','male'],[0,1],inplace=True)

In [10]:
### Cambio de datos columna Embarque ###
train['Embarked'].replace(['Q','S','C'],[0,1,2],inplace=True)
test['Embarked'].replace(['Q','S','C'],[0,1,2],inplace=True)

In [11]:
### Averiguar la media de la columna edad ###
print(train['Age'].mean())
print(test['Age'].mean())


29.69911764705882
30.272590361445783


In [12]:
### Remplazo los faltantes con la edad de la media ###
train['Age'] = train['Age'].replace(np.nan,train['Age'].mean())
test['Age'] = test['Age'].replace(np.nan,test['Age'].mean())

In [13]:
### Agrupar bandas de edaders ###
#Bandas: 0-8, 9-15, 16-18, 19-25, 26-40, 41-60, 60-100
bins = [0, 8, 15, 18, 25, 50, 60, 100]
names = ['1','2','3','4','5','6','7']
train['Age'] = pd.cut(train['Age'], bins, labels = names)
test['Age'] = pd.cut(test['Age'], bins, labels = names)

In [14]:
### Eliminacion de la columna 'Cabin' ###
train.drop(['Cabin'], axis = 1, inplace=True)
test.drop(['Cabin'], axis = 1, inplace=True)

In [15]:
### Eliminacion de las columnas que no se consideran necesarias ###
train = train.drop(['PassengerId','Name','Ticket'], axis = 1)
test = test.drop(['Name','Ticket'], axis = 1)

In [16]:
### Eliminacion de las filas con datos perdidos ###
train.dropna(axis=0, how='any', inplace=True)
test.dropna(axis=0, how='any', inplace=True)

In [17]:
##### Verificacion de datos faltantes en los dataset #####
print(pd.isnull(train).sum())
print(pd.isnull(test).sum())

print(train.shape)
print(test.shape)

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
(889, 8)
(417, 8)


## Algoritmo

* Separacion de la columna con la informacion de sobrevivientes
* Separacion de los datos de entrenamiento y prueba de 'train'

In [23]:
X = np.array(train.drop(['Survived'], 1))
y = np.array(train['Survived'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [25]:
### Regresion logistica
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
print('Presicion de Regresion Logistica:')
print(logreg.score(X_train, y_train))

Presicion de Regresion Logistica:
0.7918424753867792


In [26]:
### Support Vector Machines
svc = SVC()
svc.fit(X_train,y_train)
Y_pred = svc.predict(X_test)
print('Presicion de Soporte de Vectores:')
print(svc.score(X_train, y_train))

Presicion de Soporte de Vectores:
0.6680731364275668


In [27]:
### K Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,y_train)
Y_pred = knn.predict(X_test)
print('Presicion de Vecinos Cercanos:')
print(knn.score(X_train, y_train))

Presicion de Vecinos Cercanos:
0.8593530239099859


#### Metodo con mayor presicion: K Neighbors